In [23]:
# SEO环境： schedule
import os
import warnings
warnings.filterwarnings('ignore')
import requests
import pandas as pd
import numpy as np
import jsonpath
import json
import time
import datetime
import xlwings as xw
import telebot
import hmac, base64, struct, hashlib
from openpyxl import Workbook, load_workbook
from openpyxl import formatting, styles
from openpyxl.styles import Color, PatternFill, Font, Border
from PIL import ImageGrab
import pyperclip

pd.set_option('display.max_colwidth', None) #显示单元格完整信息
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

day = -1
pages_user = 150
pages_fircharge = 60
pages_fircharge_two= 100

# start_date = (datetime.datetime.now()+datetime.timedelta(days=day)).strftime('%Y%m%d')
# end_date = (datetime.datetime.now()+datetime.timedelta(days=day)).strftime('%Y%m%d')
# last_date = (datetime.datetime.now()+datetime.timedelta(days=day-1)).strftime('%Y%m%d')

url = 'http://fundmng.bsportsadmin.com/api/manage/fund/withdraw/record/list/history'

# 第一次获取token
submit_url = 'http://fundmng.bsportsadmin.com/api/manage/user/admin/login/submit'
header0 = {
    'Accept':'application/json, text/plain, */*',
    # 'Accept-Encoding':'gzip, deflate',
    'Accept-Language':'zh-CN,zh;q=0.9',
    'Connection':'keep-alive',
    'Content-Length':'48',
    'Content-Type':'application/x-www-form-urlencoded',
    'Cookie':'admin-uid=690; admin-token=db76bebda5274c80adaadd40bd794f24',
    'Device_id':'1.0',
    'Gl_version':'2.0',
    'Host':'fundmng.bsportsadmin.com',
    'Language':'zh_CN',
    'Origin':'http://fundmng.bsportsadmin.com',
    'Os_type':'0',
    'Referer':'http://fundmng.bsportsadmin.com/login',
    'Sign':'2bc4c378817f47731f0adf450a627d19',
    'Some':'header',
    'Systemid':"",
    'Timestamp':'1692415901000',
    'Token':'-1',
    'Uid':'-1',
    'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/116.0.0.0 Safari/537.36',
    'Version':'1.0'
}
def get_google_code(secret):
    key = base64.b32decode(secret, True)
    msg = struct.pack(">Q", int(time.time()) // 30)
    google_code = hmac.new(key, msg, hashlib.sha1).digest()
    # 很多网上的代码不可用，就在于这儿，没有chr字符串
    o = ord(chr(google_code[19])) & 15
    # google_code = (struct.unpack(">I", google_code[o:o + 4])[0] & 0x7fffffff) % 1000000
    google_code = (struct.unpack(">I", google_code[o:o + 4])[0] & 0x7fffffff) % 1000000
    return '%06d' % google_code


# 采集会员列表和会员首存记录
url_fircharge = 'http://fundmng.bsportsadmin.com/api/manage/data/detail/firstRecharge'
url_user = 'http://fundmng.bsportsadmin.com/api/manage/user/maintain/user/list'
session = requests.session()

# 1.采集首存报表
dic_fir = dict({'会员名':[], '所属代理':[],'注册时间':[], '交易时间':[], '交易类型':[], '币种':[], '金额':[]})
# 昨天开始时间戳
yesterday = datetime.date.today() + datetime.timedelta(days=day)
yesterday_start_time = int(time.mktime(time.strptime(str(yesterday), '%Y-%m-%d')))
# 昨天结束时间戳
yesterday_end_time = int(time.mktime(time.strptime(str(datetime.date.today()), '%Y-%m-%d'))) - 1

#---------------------token----------------------
# 采取token
google_code = get_google_code('64ehnxj6yily5bhv23kgb62ozuh6yuu2')
data0 = {
    'username': 'Marquis',
    'password': 'qwer123456',
    'code': google_code
}
session0 = requests.Session()
response0  =session0.post(url=submit_url,data=data0,headers=header0)
response0.encoding = 'utf-8'
obj0 = json.loads(response0.text)
token = obj0['data']['token']

header = {
    'Accept':'application/json, text/plain, */*',
    # 'Accept-Encoding':'gzip, deflate',
    'Accept-Language':'zh-CN,zh;q=0.9',
    'Connection':'keep-alive',
    'Content-Length':'75',
    'Content-Type':'application/x-www-form-urlencoded',
    'Cookie':'admin-token=67c8b1bd1b434f898ed8570a860355b8; admin-uid=690',
    'Device_id':'1.0',
    'Gl_version':'2.0',
    'Host':'fundmng.bsportsadmin.com',
    'Language':'zh_CN',
    'Menuid':'100112',
    'Opeartionmenu':'%u62A5%u8868%u67E5%u8BE2-%u4F1A%u5458%u9996%u5B58%u62A5%u8868',
    'Origin':'http://fundmng.bsportsadmin.com',
    'Os_type':'0',
    'Referer':'http://fundmng.bsportsadmin.com/system/report-query/report-first-recharge',
    'Sign':'ca83944852acc68fe114cbc65f1e1d22',
    'Some':'header',
    'Systemid':'54',
    'Timestamp':'1692092554000',
    'Token':token,
    'Uid':'690',
    'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/115.0.0.0 Safari/537.36',
    'Version':'1.0'
}
for page in range(1,pages_fircharge+1):
    data = {
        'page': page,
        'size': 20,
        'tradeType': 0,
        'isFake': 0,
        'stime': yesterday_start_time*1000,
        'etime': yesterday_end_time*1000+999
    }
    response = session.post(url_fircharge,headers=header,data=data)
    response.encoding = 'utf-8'
    obj = json.loads(response.text)

    for i in obj['data']['list']:
        dic_fir['会员名'].append(i['userName'])
        dic_fir['所属代理'].append(i['parentName'])
        dic_fir['注册时间'].append(time.strftime('%Y/%m/%d %H:%M:%S',time.localtime(i['createTime']//1000)))
        dic_fir['交易时间'].append(time.strftime('%Y/%m/%d %H:%M:%S',time.localtime(i['regTime']//1000)))
        dic_fir['交易类型'].append(i['tradeType'])
        dic_fir['币种'].append(i['coinCode'])
        dic_fir['金额'].append(i['amount'])
firChargeUser = pd.DataFrame(dic_fir)
print('会员首存行列：',firChargeUser.shape)

# 2.采集昨天和今天的首存报表
dic_fir_two = dict({'会员名':[], '所属代理':[],'注册时间':[], '交易时间':[], '交易类型':[], '币种':[], '金额':[]})
for page in range(1,pages_fircharge_two+1):
    data = {
        'page': page,
        'size': 20,
        'tradeType': 0,
        'isFake': 0,
        'stime': yesterday_start_time*1000,
        'etime': int(time.time())*1000
    }
    response = session.post(url_fircharge,headers=header,data=data)
    response.encoding = 'utf-8'
    obj = json.loads(response.text)

    for i in obj['data']['list']:
        dic_fir_two['会员名'].append(i['userName'])
        dic_fir_two['所属代理'].append(i['parentName'])
        dic_fir_two['注册时间'].append(time.strftime('%Y/%m/%d %H:%M:%S',time.localtime(i['createTime']//1000)))
        dic_fir_two['交易时间'].append(time.strftime('%Y/%m/%d %H:%M:%S',time.localtime(i['regTime']//1000)))
        dic_fir_two['交易类型'].append(i['tradeType'])
        dic_fir_two['币种'].append(i['coinCode'])
        dic_fir_two['金额'].append(i['amount'])
charge_two  = pd.DataFrame(dic_fir_two)
print('昨天与今天的会员首存行列：',charge_two.shape)

# 采集会员列表
dic_user = dict({'会员账号':[], '姓名':[],'代理':[],'玩家层级':[], '注册时间':[],'首存时间':[], '备注':[]})
for page in range(1,pages_user+1):
    data2 = {
        'page':page,
        'size':20,
        'userVip':'0,1,2,3,4,5,6,7,8,9,10,11',
        'status':'0,1,2,4',
        'sortType':'3',
        'sortStr':'descend',
        'searchType':'1',
        'channelId':'34',
        'registeredStartDate':yesterday_start_time*1000,
        'registeredEndDate':yesterday_end_time*1000+999,
    }
    response2 = session.post(url_user,headers=header,data=data2)
    response2.encoding = 'utf-8'
    obj2 = json.loads(response2.text)
    for i in obj2['data']['list']:
        dic_user['会员账号'].append(i['username'])
        if i['reallyName'] !='':
            dic_user['姓名'].append(i['reallyName'])
        else:
            dic_user['姓名'].append('-')

        dic_user['代理'].append(i['parentName'])
        dic_user['玩家层级'].append(i['levelName'])
        dic_user['注册时间'].append(time.strftime('%Y/%m/%d',time.localtime(i['registerDate']//1000)))
        dic_user['首存时间'].append(time.strftime('%Y/%m/%d',time.localtime(i['firstRechargeDate']//1000)))
        dic_user['备注'].append(i['remark'])
user = pd.DataFrame(dic_user)
print('用户列表行列:',user.shape)
# 删除测试账号
user = user[~user['会员账号'].str.contains('test')&~user['会员账号'].str.contains('ceshi')&~user['姓名'].str.contains('测试') \
            &~user['姓名'].str.contains('ceshi')&~user['代理'].str.contains('测试')&~user['代理'].str.contains('ceshi') \
            &~user['备注'].str.contains('测试')&~user['备注'].str.contains('试玩')&~user['备注'].str.contains('晒单')\
                &~user['玩家层级'].str.contains('测试')]
print('去重后：',user.shape)

#采集交易明细

user.head(2)
firChargeUser.head(3)
charge_two.head(2)

会员首存行列： (410, 7)
昨天与今天的会员首存行列： (555, 7)
用户列表行列: (1118, 7)
去重后： (1022, 7)


,会员账号,姓名,代理,玩家层级,注册时间,首存时间,备注
0,Azw9688Azw,-,gt0005,新注册默认层级,2023/10/10,1970/01/01,
1,xp0528,陈*勋,zzzb04,初审后层级,2023/10/10,2023/10/11,


,会员名,所属代理,注册时间,交易时间,交易类型,币种,金额
0,cw1206cw,btyseo7003,2023/10/10 23:54:24,2023/07/24 06:45:29,存款,CNY,300.0
1,Thienkim562,vntf0902,2023/10/10 23:49:15,2023/10/04 22:05:16,存款,VND,60000.0
2,Vanhoa82,vnzb05,2023/10/10 23:46:14,2023/10/10 21:07:32,存款,VND,600000.0


,会员名,所属代理,注册时间,交易时间,交易类型,币种,金额
0,kialos999000,gfas001,2023/10/11 14:12:59,2023/10/04 16:13:46,存款,VND,700000.0
1,Nghi130795,vnzb03,2023/10/11 14:11:26,2023/10/11 02:31:08,存款,VND,200000.0


In [24]:
daili = pd.read_excel(r'C:\Users\User\Desktop\SEO\SEO提单数据\1010\代理线.xlsx')
daili.shape

(32, 4)

In [22]:
merge_user = pd.merge(user,daili,how = 'left',left_on='代理',right_on='代理线')
result = merge_user.groupby('seo变化数据团队').agg({'seo变化数据团队':len})
result.rename(columns={'seo变化数据团队':'注册数'},inplace=True)
result

,注册数
seo变化数据团队,
aber,37
ben,5
dk,13
hugo,55
martin,62
paddy,1
tony,32
zed,4


In [6]:
merge_user_dropna = merge_user.loc[~merge_user['seo变化数据团队'].isna()&(merge_user['姓名']!='-'),]
bangka = merge_user_dropna.groupby('seo变化数据团队').agg({'会员账号':len})
bangka.rename(columns={'会员账号':'绑卡数'},inplace=True)
bangka

,绑卡数
seo变化数据团队,
DK,2
aber,14
ben,2
hugo,15
martin,22
tony,12


In [7]:
result2 = result.join(bangka).fillna(0)
result2['绑卡率'] = result2['绑卡数']/result2['注册数']
result2

,注册数,绑卡数,绑卡率
seo变化数据团队,,,
DK,13,2.0,0.153846
aber,37,14.0,0.378378
ben,5,2.0,0.400000
hugo,55,15.0,0.272727
martin,62,22.0,0.354839
paddy,1,0.0,0.000000
tony,32,12.0,0.375000
zed,4,0.0,0.000000


In [8]:
trade = pd.read_csv(r'C:\Users\User\Desktop\SEO\SEO提单数据\1010\交易明细报表1010.csv',encoding='gbk')
trade.shape

(29657, 18)

In [9]:
huiyuan = pd.read_csv(r'C:\Users\User\Desktop\SEO\SEO提单数据\1010\流失会员详情.csv',encoding='gbk')
huiyuan.shape

(956142, 21)

In [10]:
huiyuan2 = huiyuan.loc[huiyuan['首存时间']=='--',]
merge_huiyuan = huiyuan2.merge(daili,left_on = '代理',right_on='代理线')
merge_huiyuan.head()

,会员账号,代理,用户状态,用户层级,VIP等级,注册时间,首存时间,备注,绑定银行卡,同设备号会员,APP登陆,未登录天数,未下注天数,未充值天数,最后成功登录时间,最后尝试登录时间,最后存款时间,币种,充值金额,最近3个月下注金额,最近3个月公司盈利,代理线,主线账号,seo变化数据团队,准点激活团队
0,test123459,btyseo6007,正常,测试层级-1,VIP0,2022/2/18 20:26,--,NaN,否,否,否,600,600,600,2022/2/18 20:26,2022/2/18 20:26,--,NaN,0,0.0,0.0,btyseo6007,btyseo6007,hugo,SEO
1,sinuoke188,btyseo6007,正常,新注册默认层级,VIP0,2022/2/19 17:46,--,NaN,否,否,否,599,599,599,2022/2/19 17:46,2022/2/19 17:46,--,NaN,0,0.0,0.0,btyseo6007,btyseo6007,hugo,SEO
2,fxx521521,btyseo6007,正常,新注册默认层级,VIP0,2022/2/19 17:50,--,NaN,否,否,否,599,599,599,2022/2/19 17:57,2022/2/19 17:57,--,NaN,0,0.0,0.0,btyseo6007,btyseo6007,hugo,SEO
3,www1234567,btyseo6007,正常,新注册默认层级,VIP0,2022/2/18 20:39,--,NaN,否,否,否,440,600,600,2022/7/28 15:21,2022/7/28 15:21,--,NaN,0,0.0,0.0,btyseo6007,btyseo6007,hugo,SEO
4,hu130510,btyseo6007,正常,seo新注册默认层级,VIP0,2022/2/18 20:17,--,NaN,否,否,否,323,600,600,2022/11/22 22:34,2022/11/22 22:34,--,NaN,0,0.0,0.0,btyseo6007,btyseo6007,hugo,SEO


In [11]:
merge_huiyuan2 = merge_huiyuan.loc[~merge_huiyuan['seo变化数据团队'].isna(),]
merge_huiyuan2.shape

(300457, 25)

In [40]:
# charge_two = pd.read_csv(r'C:\Users\User\Desktop\SEO\SEO提单数据\1007\会员首存报表1006-1007.csv',encoding='gbk')
# charge_two.shape

(695, 7)

In [12]:
mingdan= set(list(merge_huiyuan2['会员账号'].values)+list(charge_two['会员名'].values))
len(mingdan)

300970

In [13]:
# 删除测试账号
trade2 = trade[~trade['账户名'].str.contains('test')&~trade['账户名'].str.contains('ceshi')&~trade['所属代理'].str.contains('测试')&~trade['所属代理'].str.contains('ceshi')]
trade_daili= trade2.merge(daili,left_on='所属代理',right_on='代理线')
trade_daili.shape
trade_daili.head()

(4210, 22)

,交易号,币种,账户名,账户类型,所属代理,交易类型,细分类型,操作动作,调整原因,备注,操作时间,交易金额,手续费,实际到账金额,账变前余额,账变后余额,状态,账变时间,代理线,主线账号,seo变化数据团队,准点激活团队
0,CZ202310102359572LXY,CNY,zbz8773022,会员,btyseo7002,充值,银行卡转卡,银行卡转卡,NaN,张博政||ze6wb,2023/10/10 23:59,1000.0,0.0,1000.0,0.008,1000.008,成功,2023/10/11 0:02,btyseo7002,btyseo7002,max,SEO
1,CZ202310102359572KYQ,CNY,bigbeefeng123,会员,btyseo7002,充值,银行卡转卡,银行卡转卡,NaN,陈振文||bc8qh,2023/10/10 23:59,2000.0,0.0,2000.0,0.820,2000.820,成功,2023/10/11 0:18,btyseo7002,btyseo7002,max,SEO
2,CZ202310102358572FZ0,CNY,ffjo3477,会员,btyseo7002,充值,银行卡转卡,银行卡转卡,NaN,陈嘉骏||fbzqe,2023/10/10 23:58,200.0,0.0,200.0,0.530,200.530,成功,2023/10/11 0:00,btyseo7002,btyseo7002,max,SEO
3,CZ202310102357572788,CNY,580962544h,会员,btyseo7002,充值,银行卡转卡,银行卡转卡,NaN,龙方杰||5dyd5,2023/10/10 23:57,200.0,0.0,200.0,0.008,200.008,成功,2023/10/11 0:01,btyseo7002,btyseo7002,max,SEO
4,CZ202310102356571YMN,CNY,Hukai7,会员,btyseo7002,充值,银行卡转卡,银行卡转卡,NaN,胡凯||ucvsk,2023/10/10 23:56,1000.0,0.0,1000.0,0.620,1000.620,成功,2023/10/10 23:58,btyseo7002,btyseo7002,max,SEO


In [14]:
tidan = trade_daili.loc[trade_daili['账户名'].isin(mingdan),]

In [55]:
tidan.shape

(113, 22)

In [15]:
tidan.drop_duplicates('账户名',inplace=True)
tidan_grp = tidan.groupby('seo变化数据团队').agg({'账户名':len}).rename(columns={'账户名':'提单人数'})
tidan_grp

,提单人数
seo变化数据团队,
DK,3
aber,13
ben,3
hugo,17
martin,22
tony,8
zed,3


In [18]:
import pandas as pd
shuju = pd.DataFrame({'人员':['paddy', 'tony', 'max', 'martin', 'zed', 'hugo', 'aber', 'DK', 'ben']})
shuju

SyntaxError: closing parenthesis ')' does not match opening parenthesis '{' (1994560860.py, line 2)

In [29]:
# 采集交易明细 trade
# 采取token
data0 = {
    'username': 'Marquis',
    'password': 'qwer123456',
    'code': get_google_code('64ehnxj6yily5bhv23kgb62ozuh6yuu2')
}
session0 = requests.Session()
response0  =session0.post(url=submit_url,data=data0,headers=header0)
response0.encoding = 'utf-8'
obj0 = json.loads(response0.text)
token = obj0['data']['token']
header = {
    'Accept':'application/json, text/plain, */*',
    # 'Accept-Encoding':'gzip, deflate',
    'Accept-Language':'zh-CN,zh;q=0.9',
    'Connection':'keep-alive',
    'Content-Length':'103',
    'Content-Type':'application/x-www-form-urlencoded',
    'Cookie':'admin-token=fc66d42ccb0f4ca58b184fbe5498158c; admin-uid=690',
    'Device_id':'1.0',
    'Gl_version':'2.0',
    'Host':'fundmng.bsportsadmin.com',
    'Language':'zh_CN',
    'Menuid':'100104',
    'Opeartionmenu':'%u62A5%u8868%u67E5%u8BE2-%u4F1A%u5458%u9996%u5B58%u62A5%u8868',
    'Origin':'http://fundmng.bsportsadmin.com',
    'Os_type':'0',
    'Referer':'http://fundmng.bsportsadmin.com/system/report-query/transaction-for-user',
    'Sign':'1abc87008c66bc0b4d3d7a3870442b16',
    'Some':'header',
    'Systemid':'54',
    'Timestamp':'1697016616000',
    'Token':token,
    'Uid':'690',
    'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/115.0.0.0 Safari/537.36',
    'Version':'1.0'
}
pages_trade = 150
url_trade ='http://fundmng.bsportsadmin.com/api/manage/data/balance/record/list'
dic_trade = dict({'账户名':[], '所属代理':[]})
for page in range(1,pages_trade+1):
    data2 = {
        'page':page,
        'size':500,
        'reportType':'0',
        'userType':'0',
        'dateType':'0',
        'startDate':yesterday_start_time*1000,
        'endDate':yesterday_end_time*1000+999,
        'type':'1'
    }
    response2 = session.post(url_trade,headers=header,data=data2)
    response2.encoding = 'utf-8'
    obj2 = json.loads(response2.text)
    for i in obj2['data']['list']:
        dic_trade['账户名'].append(i['username'])
        dic_trade['所属代理'].append(i['parentName'])

trade = pd.DataFrame(dic_trade)
print('trade:',trade.shape)

trade: (39178, 2)


In [ ]:
#  开始处理数据---------------------------------------------
print('开始处理shuju')
shuju = pd.DataFrame({'人员':['Paddy', 'Tony', 'Max', 'Martin', 'Zed', 'Hugo', 'Aber', 'DK', 'Ben','当日汇总'],
                      '日期':(datetime.datetime.now()+datetime.timedelta(days=day)).strftime('%Y/%m/%d'),
                      '发送IP':0,
                      '接受IP':0,
                      '对比昨天(总IP)':0,
                      '对比前3天均值(总IP)':0,
                      '对比前5天均值(总IP)':0,
                      '对比前7天均值(总IP)':0,
                      '对比昨天(总注册)':0,
                      '对比前3天均值(总注册)':0,
                      '对比前5天均值(总注册)':0,
                      '对比前7天均值(总注册)':0,
                      '对比昨天(总开户)':0,
                      '对比前3天均值(总开户)':0,
                      '对比前5天均值(总开户)':0,
                      '对比前7天均值(总开户)':0})


shuju.set_index('人员',inplace = True)

shuju['日期'] = pd.to_datetime(shuju['日期'])
shuju.insert(1,'人员2',shuju.index)
shuju['人员2']=shuju['人员2'].str.lower()

# 第1次merge前，重置索引
shuju.reset_index(inplace=True)

merge_user = pd.merge(user,daili,how = 'left',left_on='代理',right_on='代理线')
data_todaySEO = merge_user.groupby('seo变化数据团队').agg({'seo变化数据团队':len})
data_todaySEO.rename(columns={'seo变化数据团队':'注册'},inplace=True)
data_todaySEO.reset_index(inplace=True)
data_todaySEO['人员2'] = data_todaySEO['seo变化数据团队'].str.lower()
data_todaySEO.set_index('seo变化数据团队',inplace=True)

shuju=shuju.merge(data_todaySEO,on='人员2',how='left')

shuju['注册率(%)'] = round(shuju['注册']/shuju['发送IP']*100,2)

merge_charge = pd.merge(firChargeUser,daili,how='left',left_on='所属代理',right_on='代理线')
data_todayCHARGE = merge_charge.groupby('seo变化数据团队').agg({'seo变化数据团队':len})
data_todayCHARGE= data_todayCHARGE.rename(columns={'seo变化数据团队':'开户'})
data_todayCHARGE.reset_index(inplace=True)
data_todayCHARGE['seo变化数据团队']=data_todayCHARGE['seo变化数据团队'].str.lower()
data_todayCHARGE= data_todayCHARGE.rename(columns={'seo变化数据团队':'人员2'})
# 第2次merge
shuju = pd.merge(shuju,data_todayCHARGE,how='left',on='人员2')
shuju['转化率(%)'] = round(shuju['开户']/shuju['注册']*100,2)

merge_charge['注册时间']= pd.to_datetime(merge_charge['注册时间'])
merge_charge['交易时间']= pd.to_datetime(merge_charge['交易时间'])
data_today3  = merge_charge[merge_charge['注册时间'].dt.strftime('%Y/%m/%d')==merge_charge['交易时间'].dt.strftime('%Y/%m/%d')].groupby('seo变化数据团队').agg({'seo变化数据团队':len})
data_today3.rename(columns = {'seo变化数据团队':'当日注册并开户'},inplace=True)
data_today3.reset_index(inplace=True)
data_today3['seo变化数据团队'] =data_today3['seo变化数据团队'].str.lower()
data_today3.rename(columns = {'seo变化数据团队':'人员2'},inplace=True)
# 第3次merge
shuju  = pd.merge(shuju,data_today3,how='left',on='人员2')
shuju['当日注册激活率(%)'] = round(shuju['当日注册并开户']/shuju['注册']*100,2)
shuju

In [1]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"